In [1]:
# Improve by ChatGPT & GitHub Copilot
# Running the requierements.ipynb
%run /bettik/PROJECTS/pr-data-ocean/riverama/Notebooks/requierements.ipynb

# Running functions_IT.ipynb
%run /bettik/PROJECTS/pr-data-ocean/riverama/Notebooks/OSSE_borrador/IT_modes/functions_IT_modes_v1.ipynb

In [3]:
# Directory to save frames and NetCDF files
output_dir = "/bettik/PROJECTS/pr-data-ocean/riverama/Frames/IT_Modes"
nc_output_dir = "/bettik/PROJECTS/pr-data-ocean/riverama/Datos/Filtrage/ssh_it_modes/Hourly"
os.makedirs(output_dir, exist_ok=True)
os.makedirs(nc_output_dir, exist_ok=True)

# Time range for processing Internal Tides at 2014-05-09 03:30 3270
time_range = pd.date_range(start='2014-05-01T00:30:00', end='2014-07-31T23:30:00', freq='1H')
frame_counter = 2880

ds = xr.open_mfdataset("/bettik/PROJECTS/pr-data-ocean/riverama/Datos/Filtrage/ssh_it/ssh_it_*.nc", 
                       combine='nested', concat_dim='time_counter', parallel=True).load()

def process_time_step(current_date, frame_counter):
    # Select the time step using the nearest available time
    ssh_it = ds.sel(time_counter=current_date)

    # To cartesian grid
    dx = 2  # in kilometers
    ENSLAT2D, ENSLON2D, i_lat, i_lon = create_cartesian_grid(ssh_it.nav_lat[:, 0].values,
                                                             ssh_it.nav_lon[0, :].values,
                                                             dx)
    ENSLAT2D_flat = ENSLAT2D.flatten()
    ENSLON2D_flat = ENSLON2D.flatten()

    points = np.column_stack((ssh_it.nav_lat.data.flatten(), ssh_it.nav_lon.data.flatten()))
    values = ssh_it.ssh_it.data.flatten()
    target_grid = np.column_stack((ENSLAT2D_flat, ENSLON2D_flat))
    ssh_it_cart = griddata(points, values, target_grid, method='linear')

    ssh_it_cart = ssh_it_cart.reshape(ENSLAT2D.shape)
    # INTERPOLATION OF NaNs #
    x_axis = Axis(np.arange(i_lon))
    y_axis = Axis(np.arange(i_lat))

    grid = Grid2D(y_axis, x_axis, ssh_it_cart.reshape((i_lat, i_lon)))
    has_converged, filled = fill.gauss_seidel(grid)

    ssh_it_cart_filled = xr.DataArray(data=filled,
                                      dims=["y", "x"],
                                      coords=dict(y=(["y"], np.arange(i_lat)), x=(["x"], np.arange(i_lon))))
    
    nx = ssh_it_cart_filled['x'].size
    ny = ssh_it_cart_filled['y'].size

    kx = np.fft.fftfreq(3 * nx, dx)  # km
    ky = np.fft.fftfreq(3 * ny, dx)  # km
    k, l = np.meshgrid(kx, ky)
    wavenum2D = np.sqrt(k ** 2 + l ** 2)

    k1 = 0.00252
    k2 = 0.00964
    k3 = 0.02100
    k4 = 0.03100

    wavenumbers_mode_1 = [0.5 * k1, 0.5 * (k1 + k2)]  # in km
    wavenumbers_mode_2 = [0.5 * (k1 + k2), 0.5 * (k2 + k3)]  # in km
    wavenumbers_mode_3 = [0.5 * (k2 + k3), 0.5 * (k3 + k4)]  # in km

    # Apply the bandpass filters
    bandpass_mode_1 = bandpass(wavenumbers=wavenumbers_mode_1, nx=nx, ny=ny, wavenum2D=wavenum2D)
    bandpass_mode_2 = bandpass(wavenumbers=wavenumbers_mode_2, nx=nx, ny=ny, wavenum2D=wavenum2D)
    bandpass_mode_3 = bandpass(wavenumbers=wavenumbers_mode_3, nx=nx, ny=ny, wavenum2D=wavenum2D)

    # Create the spatial window
    window = create_spatial_window(nx=nx, ny=ny)

    # Parallel processing to extract modes for single time step data
    results = Parallel(n_jobs=-1, backend='multiprocessing')(delayed(extract_it_mode)(
        ssh_it_cart_filled.values, window, dx, bandpass_mode_1, bandpass_mode_2, bandpass_mode_3) for _ in range(1)
    )

    # Extract individual modes directly from the results
    ssh_it_filtered_1, ssh_it_filtered_2, ssh_it_filtered_3 = results[0]

    lon2d, lat2d = np.meshgrid(ssh_it.nav_lon[0, :].values, ssh_it.nav_lat[:, 0].values)

    # Interpolate filtered data back to original grid
    def interpolate_back(filtered_data):
        return np.array([griddata(
            np.array([ENSLAT2D.flatten(), ENSLON2D.flatten()]).T,
            filtered_data.flatten(),
            (lat2d, lon2d),
            method='linear'
        )])

    ssh_it_filtered_1_array = interpolate_back(ssh_it_filtered_1)
    ssh_it_filtered_2_array = interpolate_back(ssh_it_filtered_2)
    ssh_it_filtered_3_array = interpolate_back(ssh_it_filtered_3)

    # Create DataArrays for each mode with nav_lat, nav_lon, and time_counter dimensions
    time_counter = ssh_it.time_counter.values
    ssh_it_mode_1 = xr.DataArray(
        data=ssh_it_filtered_1_array[0][:, :, None],  # Add a new axis for time_counter
        dims=['nav_lat', 'nav_lon', 'time_counter'],
        coords={'nav_lat': ssh_it.nav_lat[:, 0].values, 'nav_lon': ssh_it.nav_lon[0, :].values, 'time_counter': [time_counter]}
    )

    ssh_it_mode_2 = xr.DataArray(
        data=ssh_it_filtered_2_array[0][:, :, None],  # Add a new axis for time_counter
        dims=['nav_lat', 'nav_lon', 'time_counter'],
        coords={'nav_lat': ssh_it.nav_lat[:, 0].values, 'nav_lon': ssh_it.nav_lon[0, :].values, 'time_counter': [time_counter]}
    )

    ssh_it_mode_3 = xr.DataArray(
        data=ssh_it_filtered_3_array[0][:, :, None],  # Add a new axis for time_counter
        dims=['nav_lat', 'nav_lon', 'time_counter'],
        coords={'nav_lat': ssh_it.nav_lat[:, 0].values, 'nav_lon': ssh_it.nav_lon[0, :].values, 'time_counter': [time_counter]}
    )

    # Create a single xarray.Dataset
    it_modes = xr.Dataset(
        {
            'ssh_it_mode_1': ssh_it_mode_1,
            'ssh_it_mode_2': ssh_it_mode_2,
            'ssh_it_mode_3': ssh_it_mode_3,
        }
    )

    # Save it_modes to NetCDF
    nc_output_file = os.path.join(nc_output_dir, f'ssh_it_modes_{frame_counter}_{current_date.strftime("%Y%m%dT%H%M%S")}.nc')
    it_modes.to_netcdf(nc_output_file)

    # Get the date for the title
    formatted_date = pd.to_datetime(current_date).strftime('%Y-%m-%d %H:%M')

    # Plotting
    fig, axs = plt.subplots(1, 3, figsize=(15, 4), subplot_kw={'projection': ccrs.PlateCarree()})

    # Titles for each subplot
    mode_titles = ['Mode 1', 'Mode 2', 'Mode 3']

    # Custom land feature with dark gray inside and black borders
    land_feature = cfeature.NaturalEarthFeature('physical', 'land', '10m',
                                                edgecolor='black', facecolor='black')

    for i, mode in enumerate(it_modes.data_vars):
        axs[i].set_extent([it_modes.nav_lon.min(), it_modes.nav_lon.max(), it_modes.nav_lat.min(), it_modes.nav_lat.max()], crs=ccrs.PlateCarree())
        im = axs[i].pcolormesh(it_modes['nav_lon'], it_modes['nav_lat'], it_modes[mode].isel(time_counter=0), cmap=cmocean.cm.balance, vmin=-0.1, vmax=0.1, transform=ccrs.PlateCarree())
        axs[i].set_title(mode_titles[i])
        fig.colorbar(im, ax=axs[i], orientation='vertical')
        axs[i].add_feature(land_feature)  # Add the custom land feature
        axs[i].add_feature(cfeature.COASTLINE, edgecolor='black')
        axs[i].set_aspect('equal')

        # Add gridlines and labels
        gl = axs[i].gridlines(draw_labels=True, linestyle='--')
        gl.top_labels = False
        gl.right_labels = False
        gl.xlabel_style = {'size': 10}
        gl.ylabel_style = {'size': 10}

    # Set the main title
    fig.suptitle(f'Internal Tides at {formatted_date}', fontsize=16)

    plt.tight_layout(rect=[0, 0, 1, 0.95])  # Adjust layout to make room for the main title
    output_file = os.path.join(output_dir, f'frame_{frame_counter}.png')
    plt.savefig(output_file)
    plt.close(fig)

    return frame_counter + 1

# Parallel processing of time steps
frame_counter = Parallel(n_jobs=-1)(delayed(process_time_step)(current_date, frame_counter + i) for i, current_date in enumerate(time_range))

print("ta da!")


/home/riverama/.conda/envs/env-stage/lib/python3.11/site-packages/joblib/parallel.py:1359: UserWarning: Multiprocessing-backed parallel loops cannot be nested, below loky, setting n_jobs=1
  n_jobs = self._backend.configure(n_jobs=self.n_jobs, parallel=self,
/home/riverama/.conda/envs/env-stage/lib/python3.11/site-packages/joblib/parallel.py:1359: UserWarning: Multiprocessing-backed parallel loops cannot be nested, below loky, setting n_jobs=1
  n_jobs = self._backend.configure(n_jobs=self.n_jobs, parallel=self,
/home/riverama/.conda/envs/env-stage/lib/python3.11/site-packages/joblib/parallel.py:1359: UserWarning: Multiprocessing-backed parallel loops cannot be nested, below loky, setting n_jobs=1
  n_jobs = self._backend.configure(n_jobs=self.n_jobs, parallel=self,
/home/riverama/.conda/envs/env-stage/lib/python3.11/site-packages/joblib/parallel.py:1359: UserWarning: Multiprocessing-backed parallel loops cannot be nested, below loky, setting n_jobs=1
  n_jobs = self._backend.configure

ta da!
